## IBM CAPSTOPNE PROJECT

### INSTALL BeautifulSoap

In [1]:
!pip install lxml

### Fetch Data

In [2]:
import pandas as pd
urlx="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

df=pd.read_html(urlx)[0]

#apply Borough requirement -
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df2 = df[~df['Borough'].str.contains('Not assigned', na=False)]

#More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
df2= df2.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

#apply Neighbourhood requirement
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
df2.Neighbourhood = df2.Neighbourhood.fillna(df2['Borough'])
df2.head(10)  

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [70]:
#apply Borough requirement
df2 = df[~df['Borough'].str.contains('Not assigned', na=False)]
df2.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
10,M9A,Etobicoke,Islington Avenue\n
11,M1B,Scarborough,Rouge\n
12,M1B,Scarborough,Malvern\n


### CHECK The Shape

In [3]:
df2.shape

(103, 3)